In [1]:
import numpy as np
# import cv2
import tifffile as tiff
from shapely.geometry import Polygon, mapping, shape, box
from skimage import io
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
# import json
from rasterio import features
import scanpy as sc
import pandas as pd
import geopandas as gpd

from matplotlib import colors

import pickle
import scipy

import seaborn as sns

from SGanalysis.SGobject import SGobject 

import matplotlib
%matplotlib inline
# matplotlib.use('Qt5Agg')
# %matplotlib qt

from sklearn.cluster import AgglomerativeClustering as AggCluster

import sys
import os

%load_ext autoreload
%autoreload 2

tools_path  = '/Users/grantkinsler/Documents/Penn/Research/SpatialBarcodes/analysis/tools/tools.py'
sys.path.append(os.path.dirname(os.path.expanduser(tools_path)))
import tools

In [2]:
# all_files = []

# # filepath  = '/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/Shared_Grant/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/'
# filepath = '/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/'

# # roi_2
# roi_name = 'roi_2'

# for roi in [1,2,3]:
#     all_files.append(f'{filepath}roi_{roi_name})
                     
# all_files.append(')
    
### roi_name: [segmentation_file, spots_file, out_path]
roi_file_paths = {
    'roi_1':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_1/segmentations/nuclei_20240604_nuclei.tiff',
                           'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_1/exports/decode_20240604.csv',
                            'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_1/exports'
                           },
                  'roi_2':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_2/segmentations/nuclei_20240529_nuclei.tiff',
                           'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_2/exports/decode_20240604.csv',
                            'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_2/exports'
                           },
                  'roi_3':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_3/segmentations/nuclei_20240604_nuclei.tiff',
                           'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_3/exports/decode_20240604.csv',
                            'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_3/exports'
                           },
                  'timezero':{'segmentation_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/segmentations/segmentation_20240513_nuclei.tiff',
                              # 'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/genes/matrix_20240517_segmentation_segmentation_20240513_cells.csv',
                              'spots_file':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/transcripts/20240517_segmentation_withRefid.csv',
                              'out_path':'/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/exports'
                              },
                  
                  }


# /roi_2

In [47]:
roi_barcodes = {}

for roi_name,files in roi_file_paths.items():

    out_path = files['out_path']

    roi_barcodes[roi_name] = pd.read_csv(f'{out_path}/cell_by_gene_matrix_dilate10_20240718_withbarcodes_atleast3.csv')
    # roi_barcodes[roi_name] = pd.read_csv(f'{out_path}/cell_by_gene_matrix_dilate10_20240718_withbarcodes_clustering_10bcs_0.4thresh.csv')

In [48]:
barcode_names = roi_barcodes['roi_1']['barcode_names']

In [49]:
barcode_names

0                  NaN
1        bc_043-bc_094
2                  NaN
3                  NaN
4                  NaN
             ...      
39626           bc_092
39627              NaN
39628    bc_044-bc_092
39629    bc_044-bc_092
39630    bc_044-bc_092
Name: barcode_names, Length: 39631, dtype: object

In [59]:
df_0

,object_id,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,...,cell_id,called_barcodes,n_called_barcodes,barcode_names,nucleus,nucleus_dilated,nucleus_centroid,center_x,center_y,area
0,15,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,10.0,...,15,"['bc_049', 'bc_060', 'bc_091']",3,bc_049-bc_060-bc_091,"POLYGON ((14550 0, 14565 3, 14567 6, 14562 12,...",POLYGON ((14543.492868987243 -7.59137848216693...,POINT (14522.496100519931 9.741623339110342),14522.496101,9.741623,3407.061178
1,5,1.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,26.0,...,5,"['bc_070', 'bc_085']",2,bc_070-bc_085,"POLYGON ((4577 0, 4594 0, 4594 2, 4653 0, 4658...",POLYGON ((4570.577170003799 -7.658596320944374...,POINT (4609.06734006734 10.72699214365881),4609.067340,10.726992,4278.583616
2,29,1.0,0.0,0.0,0.0,2.0,0.0,0.0,8.0,30.0,...,29,"['bc_010', 'bc_040', 'bc_057', 'bc_085']",4,bc_010-bc_040-bc_057-bc_085,"POLYGON ((26219 0, 26247 0, 26247 2, 26277 0, ...","POLYGON ((26213.019979067725 -8, 26212 -8, 262...",POINT (26244.96218117854 11.327616534740546),26244.962181,11.327617,3647.102290
3,22,0.0,0.0,2.0,0.0,1.0,0.0,0.0,8.0,21.0,...,22,"['bc_001', 'bc_045']",2,bc_001-bc_045,"POLYGON ((18184 0, 18275 0, 18275 2, 18280 4, ...",POLYGON ((18176.476685652982 -6.57209065321018...,POINT (18229.19056768559 15.289839883551673),18229.190568,15.289840,5582.703110
4,25,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,28.0,...,25,"['bc_009', 'bc_038', 'bc_092']",3,bc_009-bc_038-bc_092,"POLYGON ((21466 0, 21584 0, 21584 2, 21589 4, ...",POLYGON ((21458.610950548235 -6.72022913683231...,POINT (21524.157078127144 15.185363174515997),21524.157078,15.185363,6887.185309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12930,12923,5.0,0.0,6.0,0.0,7.0,0.0,0.0,15.0,40.0,...,12923,"['bc_031', 'bc_035', 'bc_060', 'bc_086']",4,bc_031-bc_035-bc_060-bc_086,"POLYGON ((30043 25522, 30055 25523, 30069 2553...",POLYGON ((29974.54227740442 25567.626971512276...,POINT (30027.786233028102 25545.22523944848),30027.786233,25545.225239,6456.565770
12931,12910,6.0,0.0,9.0,0.0,2.0,0.0,0.0,24.0,56.0,...,12910,"['bc_063', 'bc_082']",2,bc_063-bc_082,"POLYGON ((38818 25501, 38847 25501, 38870 2550...",POLYGON ((38716.895802007275 25565.83952620298...,POINT (38810.71530069674 25534.357833700036),38810.715301,25534.357834,11382.111121
12932,12926,0.0,0.0,7.0,0.0,0.0,0.0,0.0,16.0,33.0,...,12926,['bc_008'],1,bc_008,"POLYGON ((41520 25528, 41534 25528, 41550 2553...","POLYGON ((41517.05914151162 25518.44220991278,...",POINT (41527.8397668786 25546.818271268545),41527.839767,25546.818271,4644.903744
12933,12906,8.0,0.0,16.0,2.0,9.0,0.0,1.0,28.0,139.0,...,12906,"['bc_067', 'bc_071']",2,bc_067-bc_071,"POLYGON ((48718 25494, 48737 25502, 48771 2552...","POLYGON ((48628.47668565298 25566.57209065321,...",POINT (48709.80036793954 25533.531100835324),48709.800368,25533.531101,10659.583374


In [71]:
# roi_barcodes['roi_1']

many_bc_cutoff = 2

df_0 = roi_barcodes['timezero']
df_0_many = df_0[df_0['n_called_barcodes'] > many_bc_cutoff]
df_0_many = df_0_many[df_0_many.groupby('barcode_names').barcode_names.transform(len) > 0]


df_1 = roi_barcodes['roi_1']
df_1_many = df_1[df_1['n_called_barcodes'] > many_bc_cutoff]
df_1_many = df_1_many[df_1_many.groupby('barcode_names').barcode_names.transform(len) > 1]

df_2 = roi_barcodes['roi_2']
df_2_many = df_2[df_2['n_called_barcodes'] > many_bc_cutoff]
df_2_many = df_2_many[df_2_many.groupby('barcode_names').barcode_names.transform(len) > 1]

df_3 = roi_barcodes['roi_3']
df_3_many = df_3[df_3['n_called_barcodes'] > many_bc_cutoff]
df_3_many = df_3_many[df_3_many.groupby('barcode_names').barcode_names.transform(len) > 1]

In [72]:
print(len(df_0_many['barcode_names'].unique()),len(df_1_many['barcode_names'].unique()),len(df_2_many['barcode_names'].unique()),len(df_3_many['barcode_names'].unique()))

7146 862 1505 1343


In [73]:
from itertools import combinations

In [74]:
for roi in [df_1_many, df_2_many, df_3_many]:
    # print(len(roi['barcode_names'].unique()))

    intersecting_barcodes = set(df_0_many['barcode_names']).intersection(set(roi['barcode_names']))
    print(len(intersecting_barcodes))
    print(intersecting_barcodes)

for roi_a,roi_b in combinations([df_1_many, df_2_many, df_3_many],2):
    # print(len(roi['barcode_names'].unique()))

    # print()

    intersecting_barcodes = set(roi_a['barcode_names']).intersection(set(roi_b['barcode_names']))
    print(len(intersecting_barcodes))
    print(intersecting_barcodes)

17
{'bc_015-bc_047-bc_060', 'bc_077-bc_083-bc_094', 'bc_013-bc_058-bc_081', 'bc_033-bc_060-bc_085', 'bc_014-bc_066-bc_074', 'bc_001-bc_082-bc_085', 'bc_008-bc_068-bc_085', 'bc_033-bc_037-bc_073', 'bc_033-bc_060-bc_066', 'bc_047-bc_066-bc_067', 'bc_016-bc_040-bc_055', 'bc_044-bc_048-bc_060', 'bc_010-bc_033-bc_074', 'bc_041-bc_047-bc_091', 'bc_025-bc_043-bc_066', 'bc_009-bc_025-bc_095', 'bc_041-bc_083-bc_086'}
17
{'bc_022-bc_060-bc_092', 'bc_009-bc_044-bc_086', 'bc_024-bc_031-bc_087', 'bc_048-bc_060-bc_085', 'bc_014-bc_066-bc_074', 'bc_073-bc_084-bc_086', 'bc_004-bc_048-bc_060', 'bc_010-bc_060-bc_081', 'bc_013-bc_066-bc_083', 'bc_033-bc_066-bc_081', 'bc_033-bc_052-bc_068', 'bc_012-bc_021-bc_086', 'bc_047-bc_074-bc_084', 'bc_033-bc_060-bc_095', 'bc_009-bc_048-bc_081', 'bc_019-bc_086-bc_091', 'bc_031-bc_060-bc_085'}
8
{'bc_006-bc_033-bc_066', 'bc_024-bc_029-bc_044', 'bc_001-bc_049-bc_093', 'bc_002-bc_047-bc_066', 'bc_033-bc_045-bc_063', 'bc_004-bc_014-bc_025', 'bc_014-bc_019-bc_027', 'bc_0

In [75]:
df_3_many[df_3_many['barcode_names'].isin(intersecting_barcodes)]

,object_id,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,...,cell_id,called_barcodes,n_called_barcodes,barcode_names,nucleus,nucleus_dilated,nucleus_centroid,center_x,center_y,area
10778,10859,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,7.0,...,10859,"['bc_024', 'bc_059', 'bc_085']",3,bc_024-bc_059-bc_085,"POLYGON ((21431 12162, 21447 12164, 21476 1217...",POLYGON ((21421.99046687087 12157.682999087847...,POINT (21448.54752469019 12188.318865194691),21448.547525,12188.318865,5011.782223
10878,10841,4.0,0.0,0.0,0.0,1.0,0.0,0.0,11.0,49.0,...,10841,"['bc_059', 'bc_080', 'bc_085']",3,bc_059-bc_080-bc_085,"POLYGON ((20006 12151, 20042 12165, 20042 1217...",POLYGON ((19968.604441690462 12281.68228981933...,POINT (20006.610208655227 12224.222463495775),20006.610209,12224.222463,16859.122841
10897,10807,3.0,0.0,0.0,0.0,2.0,0.0,0.0,12.0,50.0,...,10807,"['bc_024', 'bc_059', 'bc_085']",3,bc_024-bc_059-bc_085,"POLYGON ((19507 12132, 19517 12155, 19530 1216...",POLYGON ((19505.03883864862 12122.194193243091...,POINT (19489.967150571567 12204.7945437969),19489.967151,12204.794544,16364.261910
10960,11044,4.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,10.0,...,11044,"['bc_024', 'bc_059', 'bc_085']",3,bc_024-bc_059-bc_085,"POLYGON ((18390 12275, 18392 12275, 18394 1228...",POLYGON ((18383.384110341394 12267.51171334474...,POINT (18363.32596956308 12298.272557682867),18363.325970,12298.272558,3761.123349
11262,11172,4.0,0.0,0.0,0.0,2.0,0.0,0.0,7.0,15.0,...,11172,"['bc_024', 'bc_059', 'bc_085']",3,bc_024-bc_059-bc_085,"POLYGON ((18824 12352, 18849 12364, 18881 1239...",POLYGON ((18814.011318622757 12351.52434850584...,POINT (18853.60240259395 12435.011428267688),18853.602403,12435.011428,13659.736741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22526,22492,1.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,12.0,...,22492,"['bc_024', 'bc_059', 'bc_085']",3,bc_024-bc_059-bc_085,"POLYGON ((24156 18214, 24158 18227, 24156 1827...","POLYGON ((24146.71523309115 18210.28609323646,...",POINT (24109.007571635833 18276.344305404426),24109.007572,18276.344305,11200.612289
22562,22646,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,...,22646,"['bc_024', 'bc_059', 'bc_085']",3,bc_024-bc_059-bc_085,"POLYGON ((24207 18295, 24215 18297, 24253 1831...",POLYGON ((24200.93956784737 18287.045682799675...,POINT (24209.591271685254 18330.239738228003),24209.591272,18330.239738,7177.505200
37153,37106,4.0,1.0,1.0,0.0,0.0,0.0,0.0,9.0,14.0,...,37106,"['bc_015', 'bc_033', 'bc_076']",3,bc_015-bc_033-bc_076,"POLYGON ((9041 25119, 9101 25124, 9115 25128, ...","POLYGON ((9015.871717494241 25140.89588523652,...",POINT (9089.60640035543 25179.778797025887),9089.606400,25179.778797,17297.457342
37637,37721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,...,37721,"['bc_015', 'bc_033', 'bc_076']",3,bc_015-bc_033-bc_076,"POLYGON ((9255 25372, 9260 25374, 9290 25412, ...",POLYGON ((9250.809418225383 25362.920406154997...,POINT (9263.271672849773 25414.294358449533),9263.271673,25414.294358,6756.591772


In [76]:
df_2_many[df_2_many['barcode_names'].isin(intersecting_barcodes)]

,object_id,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,...,cell_id,called_barcodes,n_called_barcodes,barcode_names,nucleus,nucleus_dilated,nucleus_centroid,center_x,center_y,area
41168,41133,1.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,5.0,...,41133,"['bc_024', 'bc_059', 'bc_085']",3,bc_024-bc_059-bc_085,"POLYGON ((7162 29418, 7181 29433, 7186 29443, ...",POLYGON ((7155.781178563984 29410.168891525016...,POINT (7148.834800061757 29484.156201063102),7148.834800,29484.156201,11158.728796
42507,42521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,...,42521,"['bc_059', 'bc_080', 'bc_085']",3,bc_059-bc_080-bc_085,"POLYGON ((5886 30154, 5894 30154, 5909 30165, ...","POLYGON ((5882.488765584116 30144.63670822431,...",POINT (5887.788142329452 30200.067330621405),5887.788142,30200.067331,6241.870200
43322,43312,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,9.0,...,43312,"['bc_024', 'bc_059', 'bc_085']",3,bc_024-bc_059-bc_085,"POLYGON ((7997 30578, 8016 30579, 8026 30591, ...","POLYGON ((7977 30607.982816869375, 7977 30610,...",POINT (8026.199680424656 30642.615724190226),8026.199680,30642.615724,9943.145091
44798,44746,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,44746,"['bc_059', 'bc_080', 'bc_085']",3,bc_059-bc_080-bc_085,"POLYGON ((4661 31398, 4666 31411, 4677 31423, ...",POLYGON ((4655.452998037747 31389.679497056622...,POINT (4639.386720568418 31466.746686267306),4639.386721,31466.746686,10407.830343
49959,49920,4.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,4.0,...,49920,"['bc_059', 'bc_080', 'bc_085']",3,bc_059-bc_080-bc_085,"POLYGON ((6002 34444, 6028 34453, 6058 34470, ...",POLYGON ((5993.410530257915 34438.879354576835...,POINT (6010.333797837476 34501.365745400115),6010.333798,34501.365745,10077.803468
50003,49993,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,9.0,...,49993,"['bc_059', 'bc_080', 'bc_085']",3,bc_059-bc_080-bc_085,"POLYGON ((6057 34486, 6063 34492, 6085 34501, ...","POLYGON ((6049.443109172102 34479.45069461582,...",POINT (6068.7469378827645 34540.04685525421),6068.746938,34540.046855,10411.888255
57426,57441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,57441,"['bc_015', 'bc_033', 'bc_076']",3,bc_015-bc_033-bc_076,"POLYGON ((29416 39221, 29430 39228, 29441 3924...",POLYGON ((29414.706608159322 39211.08399588814...,POINT (29409.025055502694 39257.69128607675),29409.025056,39257.691286,6872.161077
58916,58910,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0,...,58910,"['bc_015', 'bc_033', 'bc_076']",3,bc_015-bc_033-bc_076,"POLYGON ((29193 40058, 29263 40069, 29277 4007...",POLYGON ((29178.291822088457 40086.36520958179...,POINT (29240.069347513014 40104.54864296001),29240.069348,40104.548643,11200.231710
